## Step 1: Import libraries

In [5]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

## Step 2: Define a tool

In [16]:
import requests

def company_overview(symbol):
    response = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHAVANTAGE_API_KEY}")
    data = response.json()
    return data['Description']

def today_stock_data(symbol):
    response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={ALPHAVANTAGE_API_KEY}")
    data = response.json()
    return data["Time Series (Daily)"]["2025-09-30"]

## Step 3: Call a chat model normally

In [ ]:
import openai

client = openai.Client()

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is today's Stock data for IBM."},
    ]
)

print(response.output_text)

I'm unable to provide real-time stock data. To get the latest stock value for IBM, please check a reliable financial news website or a stock market app such as Yahoo Finance, Google Finance, or your brokerage platform.


In [19]:
response.__dict__

{'id': 'resp_0014275bc4a87bbd0068dcd532a5648197a047aa6f0518f4b7',
 'created_at': 1759302962.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_0014275bc4a87bbd0068dcd533adf08197a1c582ad09309a60', content=[ResponseOutputText(annotations=[], text="I'm unable to provide real-time stock data. To get the latest stock value for IBM, please check a reliable financial news website or a stock market app such as Yahoo Finance, Google Finance, or your brokerage platform.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(ef

In [20]:
response.output[0].__dict__

{'id': 'msg_0014275bc4a87bbd0068dcd533adf08197a1c582ad09309a60',
 'content': [ResponseOutputText(annotations=[], text="I'm unable to provide real-time stock data. To get the latest stock value for IBM, please check a reliable financial news website or a stock market app such as Yahoo Finance, Google Finance, or your brokerage platform.", type='output_text', logprobs=[])],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

## Step 4: Define the input schema for our tool

Lets try giving our `today_stock_data` function as a tool to our AI.

Step 1: Define the tool in a format OpenAI can understand

In [21]:
tools = [{
    "type": "function",
    "name": "today_stock_data",
    "description": "Get today's stock data.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {"type": "string"}
        },
        "required": ["symbol"],
        "additionalProperties": False
    },
    "strict": True
}]

## Step 5: Pass the tool schema over to the model

In [22]:
from pprint import pprint

input_messages = [{"role": "user", "content": "What is today's stock data for IBM?"}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools,
)

pprint(response)

Response(id='resp_07dbe44815d7a6780068dcd5f087b48193abf4fef80340ab8e', created_at=1759303152.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-nano-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"symbol":"IBM"}', call_id='call_9h0NZDXFCJiBoSbSZQHQ668W', name='today_stock_data', type='function_call', id='fc_07dbe44815d7a6780068dcd5f1ad648193bc9da42c4d33c661', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='today_stock_data', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string'}}, 'required': ['symbol'], 'additionalProperties': False}, strict=True, type='function', description="Get today's stock data.")], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_ident

In [23]:
response.__dict__

{'id': 'resp_07dbe44815d7a6780068dcd5f087b48193abf4fef80340ab8e',
 'created_at': 1759303152.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseFunctionToolCall(arguments='{"symbol":"IBM"}', call_id='call_9h0NZDXFCJiBoSbSZQHQ668W', name='today_stock_data', type='function_call', id='fc_07dbe44815d7a6780068dcd5f1ad648193bc9da42c4d33c661', status='completed')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='today_stock_data', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string'}}, 'required': ['symbol'], 'additionalProperties': False}, strict=True, type='function', description="Get today's stock data.")],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reason

In [24]:
response.output[0].__dict__

{'arguments': '{"symbol":"IBM"}',
 'call_id': 'call_9h0NZDXFCJiBoSbSZQHQ668W',
 'name': 'today_stock_data',
 'type': 'function_call',
 'id': 'fc_07dbe44815d7a6780068dcd5f1ad648193bc9da42c4d33c661',
 'status': 'completed'}

## Step 6: Format the tool call response from the LLM

In [25]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

print("Tool call: ", tool_call)
print("Arguments: ", args)

Tool call:  ResponseFunctionToolCall(arguments='{"symbol":"IBM"}', call_id='call_9h0NZDXFCJiBoSbSZQHQ668W', name='today_stock_data', type='function_call', id='fc_07dbe44815d7a6780068dcd5f1ad648193bc9da42c4d33c661', status='completed')
Arguments:  {'symbol': 'IBM'}


In [26]:
tool_call.arguments

'{"symbol":"IBM"}'

## Step 7: Pass on the tool call arguments to our tool/python function

We now need to pass on the arguments received by the model to our python function or tool

In [27]:
result = today_stock_data(args["symbol"])
result

{'1. open': '280.8800',
 '2. high': '286.0250',
 '3. low': '280.5200',
 '4. close': '282.1600',
 '5. volume': '5926924'}

## Step 8: Append the response of the tool into the message list

In [28]:
input_messages.append(tool_call)        # append model's function call message
input_messages.append({                 # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

pprint(input_messages)

[{'content': "What is today's stock data for IBM?", 'role': 'user'},
 ResponseFunctionToolCall(arguments='{"symbol":"IBM"}', call_id='call_9h0NZDXFCJiBoSbSZQHQ668W', name='today_stock_data', type='function_call', id='fc_07dbe44815d7a6780068dcd5f1ad648193bc9da42c4d33c661', status='completed'),
 {'call_id': 'call_9h0NZDXFCJiBoSbSZQHQ668W',
  'output': "{'1. open': '280.8800', '2. high': '286.0250', '3. low': "
            "'280.5200', '4. close': '282.1600', '5. volume': '5926924'}",
  'type': 'function_call_output'}]


## Step 9: Pass the message list into the model

In [29]:
response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools,
)
pprint(response_2.output_text)

("Today's stock data for IBM is as follows:\n"
 '- Opening price: $280.88\n'
 '- Highest price: $286.03\n'
 '- Lowest price: $280.52\n'
 '- Closing price: $282.16\n'
 '- Volume: 5,926,924 shares')
